In [1]:
%load_ext autoreload
%autoreload 2
import sys; import os
sys.path.append(os.path.abspath('../'))
import numpy as np
import pandas as pd
from src import string_handlers

# Data Manipulation

Before matching the [USA Spending](https://www.usaspending.gov/#/) data to the data of Chair of Financial Management and Capital Markets (which will be called the _"chair data"_ from now on), we need to manipulate both datasets to make their fields compatible with each other.

For this purpose, we have implemented handler methods. We will apply these methods to both datasets.

## USA Spending Data

In the first notebook, we combined all files into a single .csv file using `csv_handlers.extract_all_recipients()` after downloading the data from [USA Spending](https://www.usaspending.gov/#/).

Now, we will load this .csv file and process the data with the help of the `string_handlers` module.

In [2]:
usa_df = pd.read_csv("../data/all_recipients.csv", low_memory=False)

Let's check out what we have in our combined DataFrame.

In [3]:
usa_df.dtypes

recipient_duns                       object
recipient_name                       object
recipient_doing_business_as_name     object
recipient_parent_duns                object
recipient_parent_name                object
recipient_country_code               object
recipient_country_name               object
recipient_address_line_1             object
recipient_address_line_2             object
recipient_city_name                  object
recipient_state_code                 object
recipient_state_name                 object
recipient_zip_4_code                 object
recipient_congressional_district     object
recipient_phone_number               object
recipient_fax_number                 object
recipient_city_code                  object
recipient_county_code               float64
recipient_county_name                object
recipient_zip_code                  float64
recipient_zip_last_4_code           float64
recipient_foreign_city_name          object
recipient_foreign_province_name 

We start by generating a list of unique names in our dataset. These names are taken from `recipient_name`, `recipient_parent_name`, and `recipient_doing_business_as_name`.

In [4]:
uniq_names = np.concatenate([usa_df["recipient_name"].dropna().unique(),
                             usa_df["recipient_parent_name"].dropna().unique(),
                             usa_df["recipient_doing_business_as_name"].dropna().unique()
                            ])

uniq_names = np.unique(uniq_names)
print(f"We have {len(uniq_names)} unique names.")

We have 2860451 unique names.


### Process Names

Now we will process these names. While doing this, we will always keep track of (old name, new name) pairs, so that we can match the new names with old names without worrying about from which column (recipient_name/parent_name/doing_business_as_name) the name came.

We will run two methods on the names.

__1) Letter-level processing:__ Here, we try to normalize the letters in the dataset. This consists of:
* cleaning whitespace between ampersand (&) and short neighboring words, e.g. "AT & T" -> "AT&T"
* cleaning whitespace between period (.) and short neighboring words, e.g. "U. S. A." -> "U.S.A"
* replace ampersand (&) with "AND", if it is between two long words, e.g. "ZWERK & SONS FARMS" -> "ZWERK AND SONS FARMS"
* getting rid of uncommon and unnecessary characters, e.g. "INFINITEÂ¦ABM, LLC" -> "INFINITE ABM LLC"
* getting rid of continuous multiple whitespaces, e.g. "U P COMMUNITY SERVICES, INC." -> "UP COMMUNITY SERVICES, INC."

__2) Word-level processing:__ Here, we abbreviate all instances of words that are frequently abbreviated. For instance, "incorporated" is commonly abbreviated as "inc". By replacing all "incorporated"s with "inc"s, we increase our matching rates.

In [5]:
# save a record of old names
old_new_names = pd.DataFrame(uniq_names, columns=["old"])

# process names
uniq_names = string_handlers.fix_letters(uniq_names)
uniq_names = string_handlers.fix_words(uniq_names)

# match each old name with its new name
old_new_names["new"] = uniq_names

Now that we have our (old, new) name pairs, we can add columns to our dataset, representing the "clean" versions of `recipient_name`, `recipient_parent_name`, and `recipient_doing_business_as_name`.

In [6]:
usa_df['clean_recipient_name'] = pd.merge(usa_df["recipient_name"], old_new_names, right_on="old",
                                          left_on = "recipient_name", how="left")["new"].values

usa_df['clean_recipient_parent_name'] = pd.merge(usa_df["recipient_parent_name"], old_new_names, right_on="old",
                                                 left_on = "recipient_parent_name", how="left")["new"].values

usa_df['clean_recipient_doing_business_as_name'] = pd.merge(usa_df["recipient_doing_business_as_name"], 
                                                            old_new_names, right_on="old",
                                                            left_on = "recipient_doing_business_as_name",
                                                            how="left")["new"].values
usa_df.loc[usa_df.recipient_parent_duns.isna(), 'clean_recipient_parent_name'] = np.nan
usa_df.loc[usa_df.recipient_duns.isna(), 'clean_recipient_name'] = np.nan
usa_df.loc[usa_df.recipient_duns == usa_df.recipient_parent_duns, 'clean_recipient_parent_name'] = np.nan

### Format Addresses

In order to increase matching rate, and to identify better matches among multiple candidates, we use address similarity.

Address similarity is calculated on different levels. Street names, zipcodes and state codes are used in this calculation. We implemented methods to standardize street name and state name representations. 

In [7]:
addr_cols = ["recipient_address_line_1", "recipient_address_line_2"]

In [8]:
usa_df["recipient_address_line_fixed"] = string_handlers.fix_addresses(usa_df, addr_cols)

In [9]:
usa_df["recipient_state_fixed"] = usa_df["recipient_state_code"].map(string_handlers.fix_state)

Data manipulation is complete. We can now save the DataFrame so  that it can be used in the matching.

In [10]:
usa_df.to_csv("../processed/processed_usa.csv", index=False)

## Chair Data

Similar to the USA Spending data, we read, process and save the chair data.

In [12]:
chair_df = pd.read_excel("../data/company_dataset_identifier.xlsx")

In [13]:
temp = string_handlers.fix_letters(chair_df["conm"])
chair_df["clean_conm"]= string_handlers.fix_words(temp)

In [14]:
addr_cols = ["add1","add2","add3","add4"]
chair_df["add_fixed"] = string_handlers.fix_addresses(chair_df, addr_cols)

In [15]:
chair_df["state_fixed"] = chair_df["state"].map(string_handlers.fix_state)

In [16]:
chair_df.to_csv("../processed/processed_chair.csv", index=False)

After the preprocessing we continue with [matching names](./3_matching_names.ipynb).